<a href="https://colab.research.google.com/github/KimBbaoro/KEB_toy/blob/master/keb4_2_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

이거 votinglightgbm 오류가 나오는데 나는 답이 없어... 어떻게 해야하나.


In [1]:
!pip install kaggle
from google.colab import files
files.upload()

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# Permission Warning 방지
!chmod 600 ~/.kaggle/kaggle.json

!kaggle competitions download -c costa-rican-household-poverty-prediction


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Saving kaggle.json to kaggle.json
 43% 1.00M/2.33M [00:00<00:00, 5.27MB/s]
100% 2.33M/2.33M [00:00<00:00, 10.2MB/s]


In [2]:
!unzip /content/costa-rican-household-poverty-prediction.zip

Archive:  /content/costa-rican-household-poverty-prediction.zip
  inflating: codebook.csv            
  inflating: codebook.xlsx           
  inflating: sample_submission.csv   
  inflating: sample_submission.csv.zip  
  inflating: test.csv                
  inflating: test.csv.zip            
  inflating: train.csv               
  inflating: train.csv.zip           


In [152]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import lightgbm as lgb
import xgboost as xgb
from sklearn.metrics import f1_score
from joblib import Parallel, delayed
from sklearn.base import clone
from sklearn.ensemble import VotingClassifier, ExtraTreesClassifier, RandomForestClassifier
from sklearn.utils import class_weight

import warnings
warnings.filterwarnings("ignore")

In [153]:
from sklearn.preprocessing import LabelEncoder

def encode_data(df):
  df['idhogar'] = LabelEncoder().fit_transform(df["idhogar"])

def feature_importance(forest, X_train, display_results = True):
  ranked_list = []
  zero_features = []

  importances = forest.feature_importance_
  indices = np.argsort(importances)[::-1]

  if display_results:
    print("Feature ranking : ")
  
  for f in range(X_train.shape[1]):
    if display_results:
        print("%d. feature %d (%f)" % (f + 1, indices[f], 
                                       importances[indices[f]]) + " - " + 
              X_train.columns[indices[f]])
    #다 넣는거.
    ranked_list.append(X_train.columns[indices[f]])
    #중요한게 아닌거 넣는칸.
    if importances[indices[f]] == 0.0:
      zero_features.append(X_train.columns[indices[f]])
  return ranked_list, zero_features

In [154]:
from sklearn.preprocessing import LabelEncoder

# this only transforms the idhogar field, the other things this function used to do are done elsewhere
def encode_data(df):
    df['idhogar'] = LabelEncoder().fit_transform(df['idhogar'])

# plot feature importance for sklearn decision trees    
def feature_importance(forest, X_train, display_results=True):
    ranked_list = []
    zero_features = []
    
    importances = forest.feature_importances_

    indices = np.argsort(importances)[::-1]
    
    if display_results:
        # Print the feature ranking
        print("Feature ranking:")

    for f in range(X_train.shape[1]):
        if display_results:
            print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]) + " - " + X_train.columns[indices[f]])
        
        ranked_list.append(X_train.columns[indices[f]])
        
        if importances[indices[f]] == 0.0:
            zero_features.append(X_train.columns[indices[f]])
            
    return ranked_list, zero_features

In [155]:
def do_features(df):
    feats_div = [('children_fraction', 'r4t1', 'r4t3'), 
                 ('working_man_fraction', 'r4h2', 'r4t3'),
                 ('all_man_fraction', 'r4h3', 'r4t3'),
                 ('human_density', 'tamviv', 'rooms'),
                 ('human_bed_density', 'tamviv', 'bedrooms'),
                 ('rent_per_person', 'v2a1', 'r4t3'),
                 ('rent_per_room', 'v2a1', 'rooms'),
                 ('mobile_density', 'qmobilephone', 'r4t3'),
                 ('tablet_density', 'v18q1', 'r4t3'),
                 ('mobile_adult_density', 'qmobilephone', 'r4t2'),
                 ('tablet_adult_density', 'v18q1', 'r4t2'),
                ]
    
    feats_sub = [('people_not_living', 'tamhog', 'tamviv'),
                 ('people_weird_stat', 'tamhog', 'r4t3')]

    for f_new, f1, f2 in feats_div:
        df['fe_' + f_new] = (df[f1] / df[f2]).astype(np.float32)       
    for f_new, f1, f2 in feats_sub:
        df['fe_' + f_new] = (df[f1] - df[f2]).astype(np.float32)
    
    # aggregation rules over household
    aggs_num = {'age': ['min', 'max', 'mean'],
                'escolari': ['min', 'max', 'mean']
               }
    
    aggs_cat = {'dis': ['mean']}
    for s_ in ['estadocivil', 'parentesco', 'instlevel']:
        for f_ in [f_ for f_ in df.columns if f_.startswith(s_)]:
            aggs_cat[f_] = ['mean', 'count']

    # aggregation over household
    for name_, df_ in [('18', df.query('age >= 18'))]:
        df_agg = df_.groupby('idhogar').agg({**aggs_num, **aggs_cat}).astype(np.float32)
        df_agg.columns = pd.Index(['agg' + name_ + '_' + e[0] + "_" + e[1].upper() for e in df_agg.columns.tolist()])
        df = df.join(df_agg, how='left', on='idhogar')
        del df_agg

    # Drop id's
    df.drop(['Id'], axis=1, inplace=True)
    
    return df

In [156]:
def convert_OHE2LE(df): #이 함수 작동원리가 뭐지...?// 트리 모델을 위해 le로 바꾸는거
  tmp_df = df.copy(deep = True)
  for s_ in ['pared', 'piso', 'techo', 'abastagua', 'sanitario', 'energcocinar', 'elimbasu', 
               'epared', 'etecho', 'eviv', 'estadocivil', 'parentesco', 
               'instlevel', 'lugar', 'tipovivi',
               'manual_elec']:
    #행동 원소에 매뉴얼이 들어가면
    if "manual_" not in s_:
      #df 열중에서 s_로 시작하는 걸 cols_s_에 넣어라
      cols_s_ = [f_ for f_ in df.columns if f_.startswith(s_)]
    #elec이 있으면
    elif "elec" in s_:
      #cols_s_에 해당 원소들을 넣고
      cols_s = ["public", "planpri", "noelec", "coople"]
    #해당 열들이 어떤 열들이길래 그러는거지...?
    #axis = 1이면 행방향
    #각 행마다 유일한 값들을 넣겠다.
    sum_ohe = tmp_df[cols_s_].sum(axis = 1).unique()

    #0이 있으면, label encoding이 안됐으면
    if 0 in sum_ohe:
      print('The OHE in {} is incomplete. A new column will be added before label encoding'.format(s_))

      #dummy col이다.
      col_dummy = s_+"_dummy"
      #합한게 0이면 , 합한게 1이어야 하지 않음?
      
      tp_tp = tmp_df[cols_s_].copy(deep = True)
      #어떻게 생겼는지 확인용
      tmp_df[col_dummy] = (tmp_df[cols_s_].sum(axis =1)== 0).astype(np.int8)
      cols_s_.append(col_dummy)
      sum_ohe = tmp_df[cols_s_].sum(axis = 1).unique()
      if 0 in sum_ohe:
        print("The category completion did not work")
    
    tmp_cat = tmp_df[cols_s_].idxmax(axis = 1)
    tmp_df[s_+"_LE"] = LabelEncoder().fit_transform(tmp_cat).astype(np.int16)
    if "parentesco1" in cols_s_:
      cols_s_.remove("parentesco1")
    tmp_df.drop(cols_s_, axis = 1, inplace = True)
  return tmp_df, tp_tp, sum_ohe

In [157]:
def convert_OHE2LE(df):
    tmp_df = df.copy(deep=True)
    for s_ in ['pared', 'piso', 'techo', 'abastagua', 'sanitario', 'energcocinar', 'elimbasu', 
               'epared', 'etecho', 'eviv', 'estadocivil', 'parentesco', 
               'instlevel', 'lugar', 'tipovivi',
               'manual_elec']:
        if 'manual_' not in s_:
            cols_s_ = [f_ for f_ in df.columns if f_.startswith(s_)]
        elif 'elec' in s_:
            cols_s_ = ['public', 'planpri', 'noelec', 'coopele']
        sum_ohe = tmp_df[cols_s_].sum(axis=1).unique()
        #deal with those OHE, where there is a sum over columns == 0
        if 0 in sum_ohe:
            print('The OHE in {} is incomplete. A new column will be added before label encoding'
                  .format(s_))
            # dummy colmn name to be added
            col_dummy = s_+'_dummy'
            # add the column to the dataframe
            tmp_df[col_dummy] = (tmp_df[cols_s_].sum(axis=1) == 0).astype(np.int8)
            # add the name to the list of columns to be label-encoded
            cols_s_.append(col_dummy)
            # proof-check, that now the category is complete
            sum_ohe = tmp_df[cols_s_].sum(axis=1).unique()
            if 0 in sum_ohe:
                 print("The category completion did not work")
        tmp_cat = tmp_df[cols_s_].idxmax(axis=1)
        tmp_df[s_ + '_LE'] = LabelEncoder().fit_transform(tmp_cat).astype(np.int16)
        if 'parentesco1' in cols_s_:
            cols_s_.remove('parentesco1')
        tmp_df.drop(cols_s_, axis=1, inplace=True)
    return tmp_df

In [158]:
train = pd.read_csv("/content/train.csv")
test = pd.read_csv("/content/test.csv")

test_ids = test.Id

In [159]:
def process_df(df_):
  encode_data(df_)

  return do_features(df_)

train = process_df(train)
test = process_df(test)

In [160]:
train.edjefe.unique()


array(['10', '12', 'no', '11', '9', '15', '4', '6', '8', '17', '7', '16',
       '14', '5', '21', '2', '19', 'yes', '3', '18', '13', '20'],
      dtype=object)

In [161]:
train["dependency"] = np.sqrt(train["SQBdependency"])
test["dependency"] = np.sqrt(test["SQBdependency"])

train.loc[train["edjefa"] == "no", "edjefa"] = 0
train.loc[train["edjefe"] == "no", "edjefe"] = 0
test.loc[test["edjefa"] == "no", "edjefa"] = 0
test.loc[test["edjefe"]=="no", "edjefe"] = 0

#escolari가 뭐지
train.loc[(train['edjefa'] == "yes") & (train["parentesco1"] ==1), "edjefa"] = train.loc[(train["edjefa"]== "yes") & (train["parentesco1"]==1), "escolari"]
train.loc[(train['edjefe'] == "yes") & (train["parentesco1"] ==1), "edjefe"] = train.loc[(train["edjefe"]== "yes") & (train["parentesco1"]==1), "escolari"]


test.loc[(train['edjefa'] == "yes") & (test["parentesco1"] ==1), "edjefa"] = test.loc[(train["edjefa"]== "yes") & (test["parentesco1"]==1), "escolari"]
test.loc[(train['edjefe'] == "yes") & (test["parentesco1"] ==1), "edjefe"] = test.loc[(train["edjefe"]== "yes") & (test["parentesco1"]==1), "escolari"]

#배운 여자 남자
#왜 4로 채우는거지...?
train.loc[train['edjefa']=="yes", "edjefa"] = 4
train.loc[train['edjefe']== "yes", "edjefe"] = 4

test.loc[test['edjefa']=="yes", "edjefa"] = 4
test.loc[test['edjefe']== "yes", "edjefe"] = 4

train['edjefe'] = train['edjefe'].astype("int")
train['edjefa'] = train['edjefa'].astype("int")
test['edjefe'] = test['edjefe'].astype("int")
test['edjefe'] = test['edjefe'].astype("int")

train['edjef'] = np.max(train[["edjefa", "edjefa"]], axis = 1)
test["dejef"] = np.max(test[["edjefa", "edjefa"]], axis = 1)

train["v2a1"] = train['v2a1'].fillna(0)
test['v2a1'] = test['v2a1'].fillna(0)

train["v18q1"] = train['v18q1'].fillna(0)
test['v18q1'] = test['v18q1'].fillna(0)

train["rez_esc"] = train['rez_esc'].fillna(0)
test['rez_esc'] = test['rez_esc'].fillna(0)

train.loc[train.meaneduc.isnull(), "meaneduc"] = 0
train.loc[train.SQBmeaned.isnull(), "SQBmeaned"] = 0

test.loc[test.meaneduc.isnull(), "meaneduc"] = 0
test.loc[test.SQBmeaned.isnull(), "SQBmeaned"] = 0

train.loc[(train.v14a == 1) & (train.sanitario1 == 1)&(train.abastaguano == 0), "v14a"] = 0
train.loc[(train.v14a == 1) & (train.sanitario1 == 1)&(train.abastaguano == 0), "sanitario1"] = 0

test.loc[(test.v14a == 1) & (test.sanitario1 == 1)&(test.abastaguano == 0), "v14a"] = 0
test.loc[(test.v14a == 1) & (test.sanitario1 == 1)&(test.abastaguano == 0), "sanitario1"] = 0


In [162]:
def train_test_apply_func(train_, test_, func_):
  test_["Target"] = 0
  xx = pd.concat([train_, test_])

  xx_func = func_(xx)
  train_ = xx_func.iloc[:train_.shape[0], :]
  test_ = xx_func.iloc[train_.shape[0] : , :].drop("Target", axis = 1)

  del xx, xx_func
  return train_, test

In [163]:
test.head()

,v2a1,hacdor,rooms,hacapo,v14a,refrig,v18q,v18q1,r4h1,r4h2,...,agg18_instlevel5_COUNT,agg18_instlevel6_MEAN,agg18_instlevel6_COUNT,agg18_instlevel7_MEAN,agg18_instlevel7_COUNT,agg18_instlevel8_MEAN,agg18_instlevel8_COUNT,agg18_instlevel9_MEAN,agg18_instlevel9_COUNT,dejef
0,0.0,0,5,0,1,1,0,0.0,1,1,...,2.0,0.0,2.0,0.0,2.0,0.5,2.0,0.5,2.0,17.0
1,0.0,0,5,0,1,1,0,0.0,1,1,...,2.0,0.0,2.0,0.0,2.0,0.5,2.0,0.5,2.0,17.0
2,0.0,0,5,0,1,1,0,0.0,1,1,...,2.0,0.0,2.0,0.0,2.0,0.5,2.0,0.5,2.0,17.0
3,0.0,0,14,0,1,1,1,1.0,0,1,...,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0
4,175000.0,0,4,0,1,1,1,1.0,0,0,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,11.0


In [164]:
train, test = train_test_apply_func(train,test, convert_OHE2LE)

The OHE in techo is incomplete. A new column will be added before label encoding
The OHE in instlevel is incomplete. A new column will be added before label encoding
The OHE in manual_elec is incomplete. A new column will be added before label encoding


In [165]:
cols_2_ohe = ['eviv_LE', 'etecho_LE', 'epared_LE', 'elimbasu_LE', 
              'energcocinar_LE', 'sanitario_LE', 'manual_elec_LE',
              'pared_LE'] #원 핫 이코딩으로
cols_nums = ['age', 'meaneduc', 'dependency', 
             'hogar_nin', 'hogar_adul', 'hogar_mayor', 'hogar_total',
             'bedrooms', 'overcrowding']

# def convert_geo2aggs(df_):
#   #le와 기존의 것과 cols-2ohe를 합친다.
#   tmp_df = pd.concat([df_[(['lugar_LE', "idhogar"] + cols_nums)],
#                       pd.get_dummies(df_[cols_2_ohe],
#                                      columns = cols_2_ohe)], axis = 1)
#   #왜 이러는진 모르겠지만 하고
#   geo_agg = tmp_df.groupby(["lugar_LE", "idhogar"]).mean().groupby('lugar_LE').mean().astype(np.float32)
#   #새로운 열을 만든다.
#   geo_agg.columns = pd.Index(["geo_" + e for e in geo_agg.columns.tolist()])

#   del tmp_df
#   return df_.join(geo_agg, how = "left", on = "lugar_LE")

# train, test = train_test_apply_func(train, test, convert_geo2aggs)
def convert_geo2aggs(df_):
    tmp_df = pd.concat([df_[(['lugar_LE', 'idhogar']+cols_nums)],
                        pd.get_dummies(df_[cols_2_ohe], 
                                       columns=cols_2_ohe)],axis=1)

    geo_agg = tmp_df.groupby(['lugar_LE','idhogar']).mean().groupby('lugar_LE').mean().astype(np.float32)
    geo_agg.columns = pd.Index(['geo_' + e for e in geo_agg.columns.tolist()])
    
    del tmp_df
    return df_.join(geo_agg, how='left', on='lugar_LE')

# add some aggregates by geography
train, test = train_test_apply_func(train, test, convert_geo2aggs)

In [166]:
train['num_over_18'] = 0

train[train.age>=18].groupby('idhogar').transform("count")
train

,v2a1,hacdor,rooms,hacapo,v14a,refrig,v18q,v18q1,r4h1,r4h2,...,geo_manual_elec_LE_4.0,geo_pared_LE_0.0,geo_pared_LE_1.0,geo_pared_LE_2.0,geo_pared_LE_3.0,geo_pared_LE_4.0,geo_pared_LE_5.0,geo_pared_LE_6.0,geo_pared_LE_7.0,num_over_18
0,190000.0,0,3,0,1,1,0,0.0,0,1,...,0.951732,0.677456,0.009654,0.001704,0.094832,0.001704,0.157297,0.011357,0.045997,0
1,135000.0,0,4,0,1,1,1,1.0,0,1,...,0.951732,0.677456,0.009654,0.001704,0.094832,0.001704,0.157297,0.011357,0.045997,0
2,0.0,0,8,0,1,1,0,0.0,0,0,...,0.951732,0.677456,0.009654,0.001704,0.094832,0.001704,0.157297,0.011357,0.045997,0
3,180000.0,0,5,0,1,1,1,1.0,0,2,...,0.951732,0.677456,0.009654,0.001704,0.094832,0.001704,0.157297,0.011357,0.045997,0
4,180000.0,0,5,0,1,1,1,1.0,0,2,...,0.951732,0.677456,0.009654,0.001704,0.094832,0.001704,0.157297,0.011357,0.045997,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9552,80000.0,0,6,0,1,1,0,0.0,0,2,...,0.448276,0.465517,0.000000,0.004310,0.198276,0.000000,0.176724,0.000000,0.155172,0
9553,80000.0,0,6,0,1,1,0,0.0,0,2,...,0.448276,0.465517,0.000000,0.004310,0.198276,0.000000,0.176724,0.000000,0.155172,0
9554,80000.0,0,6,0,1,1,0,0.0,0,2,...,0.448276,0.465517,0.000000,0.004310,0.198276,0.000000,0.176724,0.000000,0.155172,0
9555,80000.0,0,6,0,1,1,0,0.0,0,2,...,0.448276,0.465517,0.000000,0.004310,0.198276,0.000000,0.176724,0.000000,0.155172,0


In [167]:

#18 이상 갯수 세기
train["num_over_18"] = train.loc[train.age>=18].groupby('idhogar')['idhogar'].transform("count")

#train["num_over_18"] = train[train.age>=18].groupby('idhogar').transform("count")
#묶고, 18이상 중에 최고를
train["num_over_18"] = train.groupby("idhogar")["num_over_18"].transform("max")
train["num_over_18"] = train["num_over_18"].fillna(0)

test['num_over_18'] = 0
#18 이상 갯수 세기
test["num_over_18"] = test[test.age>=18].groupby('idhogar')['idhogar'].transform("count")
#묶고, 18이상 중에 최고를
test["num_over_18"] = test.groupby("idhogar")["num_over_18"].transform("max")
test["num_over_18"] = test["num_over_18"].fillna(0)

def extract_features(df):
    df['bedrooms_to_rooms'] = df['bedrooms']/df['rooms']
    df['rent_to_rooms'] = df['v2a1']/df['rooms']
    df['tamhog_to_rooms'] = df['tamhog']/df['rooms'] # tamhog - size of the household
    df['r4t3_to_tamhog'] = df['r4t3']/df['tamhog'] # r4t3 - Total persons in the household
    df['r4t3_to_rooms'] = df['r4t3']/df['rooms'] # r4t3 - Total persons in the household
    df['v2a1_to_r4t3'] = df['v2a1']/df['r4t3'] # rent to people in household
    df['v2a1_to_r4t3'] = df['v2a1']/(df['r4t3'] - df['r4t1']) # rent to people under age 12
    df['hhsize_to_rooms'] = df['hhsize']/df['rooms'] # rooms per person
    df['rent_to_hhsize'] = df['v2a1']/df['hhsize'] # rent to household size
    df['rent_to_over_18'] = df['v2a1']/df['num_over_18']
    # some households have no one over 18, use the total rent for those
    df.loc[df.num_over_18 == 0, "rent_to_over_18"] = df[df.num_over_18 == 0].v2a1
    
extract_features(train)    
extract_features(test)   

In [168]:
needless_cols = ['r4t3', 'tamhog', 'tamviv', 'hhsize', 'v18q', 'v14a', 'agesq',
                 'mobilephone', 'female', ]

instlevel_cols = [s for s in train.columns.tolist() if "instlevel" in s]
instlevel_cols2 = [s for s in test.columns.tolist() if "instlevel" in s]

needless_cols.extend(instlevel_cols)

train = train.drop(needless_cols, axis =1)
test = test.drop(instlevel_cols2, axis = 1)

In [169]:
def split_data(train, y, sample_weight = None, households = None, test_percentage = 0.20, seed = None):
  train2 = train.copy()

  cv_hhs = np.random.choice(households, size = int(len(households) * test_percentage), replace = False)
  cv_idx = np.isin(households, cv_hhs)
  X_test = train2[cv_idx]
  y_test = y[cv_idx]

  X_train = train2[~cv_idx]
  y_train = y[~cv_idx]

  if sample_weight is not None:
    y_train_weights = sample_weight[~cv_idx]
    return X_train, y_train, X_test, y_test, y_train_weights

  return X_train, y_train, X_test, y_test


In [170]:
X = train.query('parentesco1==1')


y = X['Target'] - 1
X = X.drop(['Target'], axis=1)

np.random.seed(seed=None)

train2 = X.copy()

train_hhs = train2.idhogar

#고유한 값을 선택하고
households = train2.idhogar.unique()
#일부분만 뽑고
cv_hhs = np.random.choice(households, size=int(len(households) * 0.15), replace=False)

cv_idx = np.isin(train2.idhogar, cv_hhs)

#테스트 데이터의 나눔
X_test = train2[cv_idx]
y_test = y[cv_idx]

#훈련 데이터와
X_train = train2[~cv_idx]
y_train = y[~cv_idx]


X_train = train2
y_train = y

train_households = X_train.idhogar

In [171]:
y_train_weights = class_weight.compute_sample_weight('balanced', y_train, indices=None)


In [172]:
extra_drop_features = [
 'agg18_estadocivil1_MEAN',
 'agg18_estadocivil6_COUNT',
 'agg18_estadocivil7_COUNT',
 'agg18_parentesco10_COUNT',
 'agg18_parentesco11_COUNT',
 'agg18_parentesco12_COUNT',
 'agg18_parentesco1_COUNT',
 'agg18_parentesco2_COUNT',
 'agg18_parentesco3_COUNT',
 'agg18_parentesco4_COUNT',
 'agg18_parentesco5_COUNT',
 'agg18_parentesco6_COUNT',
 'agg18_parentesco7_COUNT',
 'agg18_parentesco8_COUNT',
 'agg18_parentesco9_COUNT',
 'geo_elimbasu_LE_4',
 'geo_energcocinar_LE_1',
 'geo_energcocinar_LE_2',
 'geo_epared_LE_0',
 'geo_hogar_mayor',
 'geo_manual_elec_LE_2',
 'geo_pared_LE_3',
 'geo_pared_LE_4',
 'geo_pared_LE_5',
 'geo_pared_LE_6',
 'num_over_18',
 'parentesco_LE',
 'rez_esc']


In [176]:
xgb_drop_cols = extra_drop_features + ["idhogar",  'parentesco1']


In [190]:
#파라미터 정하기,
# 4
opt_parameters = {'max_depth':35, 'eta':0.1, 'silent':0, 'objective':'multi:softmax', 
                  'min_child_weight': 1, 'num_class': 4, 'gamma': 2.0, 'colsample_bylevel': 0.9, 
                  'subsample': 0.84, 'colsample_bytree': 0.88, 'reg_lambda': 0.40 }
# 5
opt_parameters = {'max_depth':35, 'eta':0.15, 'silent':1, 'objective':'multi:softmax', 'min_child_weight': 2, 
                  'num_class': 4, 'gamma': 2.5, 'colsample_bylevel': 1, 'subsample': 0.95, 'colsample_bytree': 0.85, 
                  'reg_lambda': 0.35 }

def evaluate_macroF1_lgb(predictions, truth):  
    pred_labels = predictions.argmax(axis=1)
    truth = truth.get_label()
    f1 = f1_score(truth, pred_labels, average='macro')
    return ('macroF1', 1-f1) 

fit_params={"early_stopping_rounds":500,
            "eval_metric" : evaluate_macroF1_lgb, 
            "eval_set" : [(X_train,y_train), (X_test,y_test)],
            'verbose': False,
           }

#이게 뭐야...
def learning_rate_power_0997(current_iter):
    base_learning_rate = 0.1
    min_learning_rate = 0.02
    lr = base_learning_rate  * np.power(.995, current_iter)
    return max(lr, min_learning_rate)

#로그 
fit_params['verbose'] = 50


In [206]:
import lightgbm as lgb

class VotingClassifierLGBM(VotingClassifier):
    
    def fit(self, X, y, sample_weight=None, **fit_params):
        if 'early_stopping_rounds' in fit_params:
            early_stopping_rounds = fit_params['early_stopping_rounds']
            del fit_params['early_stopping_rounds']
        else:
            early_stopping_rounds = None

        if 'eval_metric' in fit_params:
            eval_metric = fit_params['eval_metric']
            del fit_params['eval_metric']
        else:
            eval_metric = None

        if 'eval_set' in fit_params:
            eval_set = fit_params['eval_set']
            del fit_params['eval_set']
        else:
            eval_set = None

        self.fit_params = fit_params
        
        estimators_ = [(name, clf) for name, clf in self.estimators]
        self.estimators_ = []
        for name, clf in estimators_:
            if isinstance(clf, lgb.LGBMModel):
                self.estimators_.append((name, clf.booster_))
            else:
                self.estimators_.append((name, clf))

        if early_stopping_rounds is not None:
            self.estimators_ = [(name, xgb_fit_with_early_stopping(clf, X, y, early_stopping_rounds, 
                                                                     eval_metric, eval_set, sample_weight,
                                                                     **self.fit_params)) 
                                if isinstance(clf, xgb.XGBModel) else (name, clf.fit(X, y, sample_weight=sample_weight, **self.fit_params))
                                for name, clf in self.estimators_]
        else:
            self.estimators_ = [(name, clf.fit(X, y, sample_weight=sample_weight, **self.fit_params))
                                for name, clf in self.estimators_]

        return self

#이거 왜 안돼....?/ ㅠㅠㅠ


In [207]:
clfs = []
for i in range(15):
    clf = xgb.XGBClassifier(random_state=217+i, n_estimators=300, learning_rate=0.15, n_jobs=4, **opt_parameters)
    
    clfs.append(('xgb{}'.format(i), clf))
    
vc = VotingClassifierLGBM(clfs, voting='soft')
del(clfs)

#Train the final model with learning rate decay
_ = vc.fit(X_train.drop(xgb_drop_cols, axis=1), y_train, sample_weight=y_train_weights, **fit_params)

clf_final = vc.estimators_[0]
#일단 넘기자...

TypeError: ignored

In [208]:
useless_features = []
drop_features = set()
counter = 0
for est in vc.estimators_:
    ranked_features, unused_features = feature_importance(est, X_train.drop(xgb_drop_cols, axis=1), display_results=False)
    useless_features.append(unused_features)
    if counter == 0:
        drop_features = set(unused_features)
    else:
        drop_features = drop_features.intersection(set(unused_features))
    counter += 1
    
drop_features

AttributeError: ignored

In [209]:
ranked_features = feature_importance(clf_final, X_train.drop(xgb_drop_cols, axis=1))


NameError: ignored

In [210]:
ets = []    
for i in range(10):
    rf = RandomForestClassifier(max_depth=None, random_state=217+i, n_jobs=4, n_estimators=700, min_impurity_decrease=1e-3, min_samples_leaf=2, verbose=0, class_weight="balanced")
    ets.append(('rf{}'.format(i), rf))   

vc2 = VotingClassifierLGBM(ets, voting='soft')    
_ = vc2.fit(X_train.drop(et_drop_cols, axis=1), y_train, threshold=False)    

NameError: ignored

In [211]:
vc2.voting = 'soft'
global_rf_score_soft = f1_score(y_test, vc2.predict(X_test.drop(et_drop_cols, axis=1)), average='macro')
vc2.voting = 'hard'
global_rf_score_hard = f1_score(y_test, vc2.predict(X_test.drop(et_drop_cols, axis=1)), average='macro')

print('Validation score of a VotingClassifier on 3 LGBMs with soft voting strategy: {:.4f}'.format(global_rf_score_soft))
print('Validation score of a VotingClassifier on 3 LGBMs with hard voting strategy: {:.4f}'.format(global_rf_score_hard))

NameError: ignored

In [212]:
vc2.voting = 'soft'
global_rf_score_soft = f1_score(y_test, vc2.predict(X_test.drop(et_drop_cols, axis=1)), average='macro')
vc2.voting = 'hard'
global_rf_score_hard = f1_score(y_test, vc2.predict(X_test.drop(et_drop_cols, axis=1)), average='macro')

print('Validation score of a VotingClassifier on 3 LGBMs with soft voting strategy: {:.4f}'.format(global_rf_score_soft))
print('Validation score of a VotingClassifier on 3 LGBMs with hard voting strategy: {:.4f}'.format(global_rf_score_hard))

NameError: ignored

In [213]:
useless_features = []
drop_features = set()
counter = 0
for est in vc2.estimators_:
    ranked_features, unused_features = feature_importance(est, X_train.drop(et_drop_cols, axis=1), display_results=False)
    useless_features.append(unused_features)
    if counter == 0:
        drop_features = set(unused_features)
    else:
        drop_features = drop_features.intersection(set(unused_features))
    counter += 1
    
drop_features

AttributeError: ignored

In [214]:
def combine_voters(data, weights=[0.5, 0.5]):
    # do soft voting with both classifiers
    vc.voting="soft"
    vc1_probs = vc.predict_proba(data.drop(xgb_drop_cols, axis=1))
    vc2.voting="soft"
    vc2_probs = vc2.predict_proba(data.drop(et_drop_cols, axis=1))
    
    final_vote = (vc1_probs * weights[0]) + (vc2_probs * weights[1])
    predictions = np.argmax(final_vote, axis=1)
    
    return predictions

In [215]:
combo_preds = combine_voters(X_test, weights=[0.5, 0.5])
global_combo_score_soft = f1_score(y_test, combo_preds, average='macro')
global_combo_score_soft

AttributeError: ignored

In [216]:
combo_preds = combine_voters(X_test, weights=[0.4, 0.6])
global_combo_score_soft= f1_score(y_test, combo_preds, average='macro')
global_combo_score_soft

AttributeError: ignored

In [217]:
combo_preds = combine_voters(X_test, weights=[0.6, 0.4])
global_combo_score_soft = f1_score(y_test, combo_preds, average='macro')
global_combo_score_soft

AttributeError: ignored